In [1]:
import os
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import seaborn as sns 
import tensorflow as tf
import tensorflow 
from tensorflow import keras
import segmentation_models as sm
from segmentation_models import Unet
import glob

Segmentation Models: using `keras` framework.


In [2]:
CLASSES = ['road',
        'sidewalk',
        'construction',
        'tram-track',
        'fence',
        'pole',
        'traffic-light',
        'traffic-sign',
        'vegetation',
        'terrain',
        'sky',
        'human',
        'rail-track',
        'car',
        'truck',
        'trackbed',
        'on-rails',
        'rail-raised',
        'rail-embedded']

In [3]:
#these codes are used to loop through the directory
path_images = 'rs19_val/uint8/rs19_val'

breaker = 0

list_tram=[]
list_tram_w_veg=[]

list_train=[]
list_train_w_veg=[]


for filename in os.listdir(path_images):
    #breaker +=1
    if filename.endswith('.png'):    
        mask = cv2.imread('rs19_val/uint8/rs19_val/'+filename) 
        labels = np.unique(mask[:,:,0]) 
        if (3 in labels) and (13 in labels):
            list_tram.append(filename)
            if 8 in labels:
                list_tram_w_veg.append(filename)
        else:
            list_train.append(filename)
            if 8 in labels:
                list_train_w_veg.append(filename)

    # if len(list_tram) == 10:
    #     break

In [10]:
#Resizing images is optional, CNNs are ok with large images
SIZE_X = 256 #Resize images (height  = X, width = Y)
SIZE_Y = 256

In [12]:
for i in glob.glob(list_tram):
    for img_path in glob.glob(os.path.join(list_tram, "*.png")):
        #print(img_path)      
        img = cv2.resize(img, (SIZE_Y, SIZE_X))

TypeError: expected str, bytes or os.PathLike object, not list

In [6]:
#Convert list to array for machine learning processing        
tram_images = np.array(list_tram)
tram_images_veg = np.array(list_tram_w_veg)

train_images = np.array(list_train)
train_images_veg = np.array(list_train_w_veg)

In [7]:
train_images.shape

(7024,)

In [8]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)




TypeError: expected str, bytes or os.PathLike object, not ndarray